# Applying Fine-tuned Summarisation Model to HR data

__AIM:__
Provide contextual summaries of negative comments per category made by respondents during the Feb 2022 Your Say Survey by application of a Transformer model fine-tuned on Summarisation task.

In [1]:
import torch
import pandas as pd
pd.set_option('display.max_columns', None)
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
import re
from tqdm import tqdm
from IPython.display import display, HTML

In [2]:
ys_data = pd.read_csv('../yoursay_02_2022.csv')

In [3]:
ys_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80914 entries, 0 to 80913
Columns: 104 entries, ResponseId to Workforce 20/20
dtypes: object(104)
memory usage: 64.2+ MB


In [4]:
ys_data.shape

(80914, 104)

In [5]:
ys_data.head()

,ResponseId,Q1,Q4,Q6,Q7,Q8,Q9,Q10,Q11,Q14,Q15,Q16,Q17,Q18,Q20,Q21,Q22,Q23,Q24,Q26,Q27,Q28,Q30,Q30_10_TEXT,Q31,Q33,Q34,Q35,Q37,Q38,Q38_9_TEXT,Q39,Q41,Q42,Q43,Q44,Q45,Q46,Q48,Q49,Q50,Q53,Q54,Q55,Q56,Q57,Q58,Q59,Q60,Q61,Q62,Q63,Q64,Q65,Q67,Q68,Q69,Q71,Q75,Q77,Q79,Q80,Q81,Q82,Q83,Q84,Q85,Q87,Q108,Accelerate,Benefit Band,Beyond Today,Business Unit,Career Level,City,Country,Desk/Field,EE ESN - BAFO,EE ESN - Leadership & Delivery,EE GGS,Elevate,Employee Subgroup,Employee Type,Full Time / Part Time,Geography,High Potential,Level up Ethnicity,Level up Tech Woman,Location,Manager / Non Manager,Managerial Level,Mapped Band,Mapped band 2,N-Level,Organisation,Region,Regular / Non Regular,Reimagine,SMT/SLT,Strategic Location,Time in Service,UK Region,Work Mode,Workforce 20/20
0,Response ID,Making the most of your survey feedback\n\n \n...,How often do you interact with external/end cu...,I'm proud to work for [Field-Brand],I would recommend [Field-Brand] as a great pla...,I'm proud of the service we give to our customers,I recommend [Field-Brand]'s products and services,I'm excited about [Field-Brand]'s future,I feel motivated to help [Field-Brand] grow,Our senior leaders inspire me about the future...,Our senior leaders are great role models,I believe change happens for the right busines...,You answered positively to the question 'Our s...,You answered negatively to the question 'Our s...,My performance is assessed fairly,My manager values a wide range of perspectives...,My manager helps me understand how my work con...,My manager always prioritises that we deliver ...,My manager gives me regular feedback that help...,Getting things done here is straightforward,I have the authority I need to make decisions,I feel I can perform at my best on most days,You answered negatively to the question 'Getti...,You answered negatively to the question 'Getti...,Please use this comments box to tell us more a...,In [Field-Brand] people from different backgro...,I feel a sense of belonging at [Field-Brand],At [Field-Brand] I feel diversity and inclusio...,How long do you think you will continue to wor...,As part of our promise to do the best for coll...,As part of our promise to do the best for coll...,Please use this comments box to tell us more a...,I feel change is managed well in my Customer F...,I believe [Field-Brand] values my wellbeing,My manager cares about me as a person,"Overall, how satisfied are you with your life ...",Colleagues' overall wellbeing matters to us an...,Colleagues' overall wellbeing matters to us an...,I get to use my skills and abilities every day,I have the opportunity to progress my career,"In the last year, Iâ€™ve had the opportunity t...",My manager makes it clear that how I do my job...,Our senior leaders never reward or ignore unet...,"I can challenge my managers approach, ideas an...",We donâ€™t knowingly report false data,"In my team we follow the rules, policies and l...",In my team we take care to make sure our custo...,In my team we have honest conversations,In my team we compete fairly,We do business ethically,I feel able to speak up,I feel comfortable raising risks in [Field-Brand],"Where I work, people live by the 10 promises i...","Ethics is at the heart of everything we do, so...",I have the support I need to do the right thin...,Our processes help me to provide great custome...,What one thing within [Field-Brand] would you ...,I believe [Field-Brand] cares about my persona...,I feel that [Field-Brand] is managing the res...,What one thing would you change that could imp...,Within Global I believe we are transforming th...,I understand how my work supports our focus on...,I feel my manager has communicated change impa...,Making it personal and local for customers and...,I understand the transformation changes taking...,"I am clear about the priorities for Digital, a...",I feel able to be fearless (experiment and try...,Thank you for completing Section 1 of the Your...,Thank you for taking the time to share your

The first row contains details of the header and will be dropped.

In [6]:
ys_data.drop([0], inplace=True)

In [7]:
ys_data.head()

,ResponseId,Q1,Q4,Q6,Q7,Q8,Q9,Q10,Q11,Q14,Q15,Q16,Q17,Q18,Q20,Q21,Q22,Q23,Q24,Q26,Q27,Q28,Q30,Q30_10_TEXT,Q31,Q33,Q34,Q35,Q37,Q38,Q38_9_TEXT,Q39,Q41,Q42,Q43,Q44,Q45,Q46,Q48,Q49,Q50,Q53,Q54,Q55,Q56,Q57,Q58,Q59,Q60,Q61,Q62,Q63,Q64,Q65,Q67,Q68,Q69,Q71,Q75,Q77,Q79,Q80,Q81,Q82,Q83,Q84,Q85,Q87,Q108,Accelerate,Benefit Band,Beyond Today,Business Unit,Career Level,City,Country,Desk/Field,EE ESN - BAFO,EE ESN - Leadership & Delivery,EE GGS,Elevate,Employee Subgroup,Employee Type,Full Time / Part Time,Geography,High Potential,Level up Ethnicity,Level up Tech Woman,Location,Manager / Non Manager,Managerial Level,Mapped Band,Mapped band 2,N-Level,Organisation,Region,Regular / Non Regular,Reimagine,SMT/SLT,Strategic Location,Time in Service,UK Region,Work Mode,Workforce 20/20
1,R_2tJAVuw1MPzxuIC,I understand,Once every 2 weeks,Agree,Agree,Agree,Agree,Neither agree nor disagree,Neither agree nor disagree,Disagree,Neither agree nor disagree,Neither agree nor disagree,NaN,NaN,Agree,Agree,Agree,Agree,Agree,Disagree,Neither agree nor disagree,Agree,Accountability,NaN,NaN,Agree,Neither agree nor disagree,Neither agree nor disagree,Up to a year,Other,NaN,Better Work Place programme has moved my Job a...,Neither agree nor disagree,Neither agree nor disagree,Agree,Mostly,NaN,NaN,Agree,Neither agree nor disagree,Neither agree nor disagree,Strongly agree,Disagree,Agree,Strongly agree,Strongly agree,NaN,Agree,Agree,Neither agree nor disagree,Agree,Strongly agree,Agree,NaN,Agree,Agree,NaN,Agree,Agree,NaN,NaN,NaN,NaN,NaN,Neither agree nor disagree,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I confirm the above",No,EX,No,Group Human Resources,E,London,United Kingdom,NaN,No,No,0.0,No,NaN,Regular Employee,Full Time,UK,No,No,No,GBR London - Faraday Building,Manager,First Line Managers,Career Level E,E,N-6,Corporate Functions,United Kingdom,Regular,No,NaN,NaN,25+ years,London,Agile Office Worker,BT Group Standard Reward Framework UK
2,R_2PtGtcxYU8y0lFQ,I understand,Never,Agree,Agree,Strongly agree,Agree,Disagree,Disagree,Strongly agree,Agree,Agree,Keep us up to date with what is going on in th...,NaN,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Agree,Strongly agree,Strongly agree,NaN,NaN,NaN,Strongly agree,Agree,Agree,Up to a year,Location/Better Workplace Programme,NaN,Better Workplace is good for the business but ...,Strongly agree,Strongly agree,Strongly agree,Mostly,NaN,NaN,Strongly agree,Agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,Strongly agree,Strongly agree,NaN,Strongly agree,Strongly agree,Letting people continue to work from home,NaN,NaN,NaN,NaN,Agree,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I confirm the above",No,NaN,No,Group Human Resources,NaN,Milton Keynes,United Kingdom,NaN,No,No,0.0,No,NaN,Regular Employee,Full Time,UK,No,No,No,GBR Milton Keyn-Bltchly CC Adm,Non Manager,NaN,NaN,0.0,N-6,Corporate Functions,United Kingdom,Regular,No,NaN,NaN,20-25 years,South East,Fixed Office Worker,BT Group Standard Team Member Framework
3,R_2AZMxd3T1xWz4nD,I understand,Never,Agree,Neither agree nor disagree,Agree,Disagree,Neither agree nor disagree,Agree,Strongly agree,Agree,Agree,Lead by example\nClear about why changes are h...,NaN,Agree,Strongly agree,Agree,Strongly agree,Neither agree nor disagree,Disagree,Agree,Agree,IT Systems,NaN,There is always something that doesn't work qu...,Agree,Disagree,Agree,1 - 2 years,Location/Better Workplace Programme,NaN,Location may impact my work/life balance and I...,Agree,Neither agree nor disagree,Agree,A little,NaN,NaN,Agree,Disagree,Agree,Agree,Agree,Neither agree nor disagree,Agree,Strongly agree,NaN,Agree,Agree,Agree,Agree,Agree,Agree,NaN,Agree,Neither agree nor disagree,NaN,Agree,Agree,NaN,NaN,NaN,NaN,NaN,Agree,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I confirm the above"

Column Q30 represents the category field. There are a total of 12 categories excluding the blanks (NaNs)

In [8]:
ys_data['Q30'].unique()

array(['Accountability', nan, 'IT Systems', 'Collaboration',
       'Processes and procedures', 'Working environment & culture',
       'Communication', 'Leadership support', 'Development in role',
       'Working relationships', 'Other', 'Manager support',
       'Wellbeing support'], dtype=object)

In [11]:
ys_categories = list(ys_data['Q30'].unique())
ys_categories = [x for x in ys_categories if str(x) != 'nan'] #remove nan from the list
ys_categories

['Accountability',
 'IT Systems',
 'Collaboration',
 'Processes and procedures',
 'Working environment & culture',
 'Communication',
 'Leadership support',
 'Development in role',
 'Working relationships',
 'Other',
 'Manager support',
 'Wellbeing support']

#### Accountability category

In [16]:
ys_accountability = ys_data[ys_data['Q30'] == 'Accountability']

The comments from respondents were for the following statement: 

__*"You answered negatively to the question 'Getting things done here is straightforward'. What would need to change at [Field-Brand] to make getting things done more straightforward? Pick your top reason from the category boxes below. - Other - Text"*__

In [17]:
ys_accountability.head()

,ResponseId,Q1,Q4,Q6,Q7,Q8,Q9,Q10,Q11,Q14,Q15,Q16,Q17,Q18,Q20,Q21,Q22,Q23,Q24,Q26,Q27,Q28,Q30,Q30_10_TEXT,Q31,Q33,Q34,Q35,Q37,Q38,Q38_9_TEXT,Q39,Q41,Q42,Q43,Q44,Q45,Q46,Q48,Q49,Q50,Q53,Q54,Q55,Q56,Q57,Q58,Q59,Q60,Q61,Q62,Q63,Q64,Q65,Q67,Q68,Q69,Q71,Q75,Q77,Q79,Q80,Q81,Q82,Q83,Q84,Q85,Q87,Q108,Accelerate,Benefit Band,Beyond Today,Business Unit,Career Level,City,Country,Desk/Field,EE ESN - BAFO,EE ESN - Leadership & Delivery,EE GGS,Elevate,Employee Subgroup,Employee Type,Full Time / Part Time,Geography,High Potential,Level up Ethnicity,Level up Tech Woman,Location,Manager / Non Manager,Managerial Level,Mapped Band,Mapped band 2,N-Level,Organisation,Region,Regular / Non Regular,Reimagine,SMT/SLT,Strategic Location,Time in Service,UK Region,Work Mode,Workforce 20/20
1,R_2tJAVuw1MPzxuIC,I understand,Once every 2 weeks,Agree,Agree,Agree,Agree,Neither agree nor disagree,Neither agree nor disagree,Disagree,Neither agree nor disagree,Neither agree nor disagree,NaN,NaN,Agree,Agree,Agree,Agree,Agree,Disagree,Neither agree nor disagree,Agree,Accountability,NaN,NaN,Agree,Neither agree nor disagree,Neither agree nor disagree,Up to a year,Other,NaN,Better Work Place programme has moved my Job a...,Neither agree nor disagree,Neither agree nor disagree,Agree,Mostly,NaN,NaN,Agree,Neither agree nor disagree,Neither agree nor disagree,Strongly agree,Disagree,Agree,Strongly agree,Strongly agree,NaN,Agree,Agree,Neither agree nor disagree,Agree,Strongly agree,Agree,NaN,Agree,Agree,NaN,Agree,Agree,NaN,NaN,NaN,NaN,NaN,Neither agree nor disagree,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I confirm the above",No,EX,No,Group Human Resources,E,London,United Kingdom,NaN,No,No,0.0,No,NaN,Regular Employee,Full Time,UK,No,No,No,GBR London - Faraday Building,Manager,First Line Managers,Career Level E,E,N-6,Corporate Functions,United Kingdom,Regular,No,NaN,NaN,25+ years,London,Agile Office Worker,BT Group Standard Reward Framework UK
5,R_1jTJW78imCoaM8U,I understand,Once every 2 weeks,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,"Lead by example, share visions, communicate re...",NaN,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Disagree,Strongly agree,Strongly agree,Accountability,NaN,We need clearer R&R / paths to achieve the des...,Strongly agree,Strongly agree,Strongly agree,Haven't thought about it,NaN,NaN,NaN,Agree,Strongly agree,Strongly agree,Mostly,NaN,NaN,Strongly agree,Disagree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,Strongly agree,Disagree,The procurement process and getting IT equipme...,Strongly agree,Strongly agree,Better Workplace is noticeably impacting team ...,NaN,NaN,NaN,NaN,Strongly agree,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I confirm the above",No,EX,No,Group Human Resources,E,Birmingham,United Kingdom,NaN,No,No,0.0,No,NaN,Regular Employee,Full Time,UK,No,No,No,GBR-Birmingham-Three Snowhill,Manager,First Line Managers,Career Level E,E,N-5,Corporate Functions,United Kingdom,Regular,No,NaN,Strategic,2-5 years,West Midlands,Fixed Office Worker,UK Manager People Framework
12,R_3lum6LkZuLTdX6D,I understand,Never,Agree,Agree,Neither agree nor disagree,Agree,Agree,Agree,Agree,Agree,Agree,Provide clarity of vision and strategy.,NaN,Agree,Agree,Agree,Agree,Agree,Disagree,Agree,Disagree,Accountability,NaN,Clear accountabilities within and across funct...,Neither agree nor disagree,Agree,Agree,Up to a year,Personal reasons,NaN,NaN,Neither agree nor disagree,Strongly agree,Strongly agree,Mostly,NaN,NaN,Strongly agree,Agree,Strongly agree,Strongly agree,Agree,Agree,Strongly agree,Strongly agree,NaN,Agree,Agree,Strongly agree,Agree,Strongly agree,Agree,NaN,Agree,Disagree,end to end processes/service support,Strongly agree,Strongly agree,quicker

In [18]:
## remove NaNs from the comments
ys_accountability = ys_accountability[~ys_accountability['Q31'].isnull()]

In [19]:
ys_accountability.head()

,ResponseId,Q1,Q4,Q6,Q7,Q8,Q9,Q10,Q11,Q14,Q15,Q16,Q17,Q18,Q20,Q21,Q22,Q23,Q24,Q26,Q27,Q28,Q30,Q30_10_TEXT,Q31,Q33,Q34,Q35,Q37,Q38,Q38_9_TEXT,Q39,Q41,Q42,Q43,Q44,Q45,Q46,Q48,Q49,Q50,Q53,Q54,Q55,Q56,Q57,Q58,Q59,Q60,Q61,Q62,Q63,Q64,Q65,Q67,Q68,Q69,Q71,Q75,Q77,Q79,Q80,Q81,Q82,Q83,Q84,Q85,Q87,Q108,Accelerate,Benefit Band,Beyond Today,Business Unit,Career Level,City,Country,Desk/Field,EE ESN - BAFO,EE ESN - Leadership & Delivery,EE GGS,Elevate,Employee Subgroup,Employee Type,Full Time / Part Time,Geography,High Potential,Level up Ethnicity,Level up Tech Woman,Location,Manager / Non Manager,Managerial Level,Mapped Band,Mapped band 2,N-Level,Organisation,Region,Regular / Non Regular,Reimagine,SMT/SLT,Strategic Location,Time in Service,UK Region,Work Mode,Workforce 20/20
5,R_1jTJW78imCoaM8U,I understand,Once every 2 weeks,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,"Lead by example, share visions, communicate re...",NaN,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Disagree,Strongly agree,Strongly agree,Accountability,NaN,We need clearer R&R / paths to achieve the des...,Strongly agree,Strongly agree,Strongly agree,Haven't thought about it,NaN,NaN,NaN,Agree,Strongly agree,Strongly agree,Mostly,NaN,NaN,Strongly agree,Disagree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,NaN,Strongly agree,Disagree,The procurement process and getting IT equipme...,Strongly agree,Strongly agree,Better Workplace is noticeably impacting team ...,NaN,NaN,NaN,NaN,Strongly agree,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I confirm the above",No,EX,No,Group Human Resources,E,Birmingham,United Kingdom,NaN,No,No,0.0,No,NaN,Regular Employee,Full Time,UK,No,No,No,GBR-Birmingham-Three Snowhill,Manager,First Line Managers,Career Level E,E,N-5,Corporate Functions,United Kingdom,Regular,No,NaN,Strategic,2-5 years,West Midlands,Fixed Office Worker,UK Manager People Framework
12,R_3lum6LkZuLTdX6D,I understand,Never,Agree,Agree,Neither agree nor disagree,Agree,Agree,Agree,Agree,Agree,Agree,Provide clarity of vision and strategy.,NaN,Agree,Agree,Agree,Agree,Agree,Disagree,Agree,Disagree,Accountability,NaN,Clear accountabilities within and across funct...,Neither agree nor disagree,Agree,Agree,Up to a year,Personal reasons,NaN,NaN,Neither agree nor disagree,Strongly agree,Strongly agree,Mostly,NaN,NaN,Strongly agree,Agree,Strongly agree,Strongly agree,Agree,Agree,Strongly agree,Strongly agree,NaN,Agree,Agree,Strongly agree,Agree,Strongly agree,Agree,NaN,Agree,Disagree,end to end processes/service support,Strongly agree,Strongly agree,quicker decision making,NaN,NaN,NaN,NaN,Strongly agree,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I confirm the above",No,CX,No,Group Human Resources,C,Bristol,United Kingdom,NaN,No,No,0.0,No,NaN,Regular Employee,Full Time,UK,No,No,No,GBR-Bristol-Temple Point,Manager,Middle Managers,Career Level C,C,N-4,Corporate Functions,United Kingdom,Regular,No,NaN,NaN,25+ years,South West,Mobile Office Worker,BT Group Standard Reward Framework UK
142,R_3nJDykPG41tC3Co,I understand,Every Day,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,Strongly agree,The Senior area manager we have had over the l...,NaN,Neither agree nor disagree,Agree,Agree,Agree,Neither agree nor disagree,Disagree,Disagree,Neither agree nor disagree,Accountability,NaN,accountability on other departments and teams....,Agree,Agree,Agree,5+ years,NaN,NaN,NaN,Agree,Agree,Agree,Somewhat,NaN,NaN,Agree,Agree,Agree,Agree,Agree,Agree,Neither agree nor disagree,Agree,NaN,Agree,Agree,Agree,Agree,Agree,Agree,NaN,Agree,Agree,NaN,Agree,Agree,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I confirm I am happy to submit my responses to...,"Yes, I con

In [20]:
ys_accountability_corpus = list(ys_accountability['Q31'])

In [21]:
len(ys_accountability_corpus)

1169

We can view the first 10 comments within accountability

In [25]:
list(ys_accountability['Q31'].head(10))

["We need clearer R&R / paths to achieve the desired outcome. In general it's not great at BT. Good example however of where this is done well is the Privacy Impact Assessment Process.",
 'Clear accountabilities within and across functions would enable greater speed of change/delivery',
 'accountability on other departments and teams. Always down to the SD manager to chase, feedback and challenged on things that are out of their control. ',
 'Getting things done requires too many sign offs and too many hoops to jump through. Itâ€™s not clear how our work is valued across the organisation.',
 'Decisions are made at a senior level without proper thought of how we actually achieve the goals set with the resource provided. It then becomes the blame game when it goes wrong. There needs to be more discussion around fundamental decisions with the boots on the ground.',
 "Quite often processes fall to the same people as not enough people have the knowledge/ experience or time to take any respo


<br>
We need to clean the text to remove new line characters and tags

In [13]:
def clean_text(text):
    
    # lowercase
    #text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    # Only keep:
    # Letters a-z, A-Z
    # Dots (.)
    # Numbers (\d)
    # Non white space (\s)

    text = re.sub('[^a-zA-Z.\d\s]', '', text)
    text = re.sub("\n", '', text) # remove new line character

    
    return text

In [40]:
## clean text
ys_accountability['Q31_cleaned'] = ys_accountability['Q31'].apply(lambda x:clean_text(x))

In [41]:
list(ys_accountability['Q31'].head(10))

["We need clearer R&R / paths to achieve the desired outcome. In general it's not great at BT. Good example however of where this is done well is the Privacy Impact Assessment Process.",
 'Clear accountabilities within and across functions would enable greater speed of change/delivery',
 'accountability on other departments and teams. Always down to the SD manager to chase, feedback and challenged on things that are out of their control. ',
 'Getting things done requires too many sign offs and too many hoops to jump through. Itâ€™s not clear how our work is valued across the organisation.',
 'Decisions are made at a senior level without proper thought of how we actually achieve the goals set with the resource provided. It then becomes the blame game when it goes wrong. There needs to be more discussion around fundamental decisions with the boots on the ground.',
 "Quite often processes fall to the same people as not enough people have the knowledge/ experience or time to take any respo

## Apply the pre-trained model

We will load the pre-trained model, the tokenizer and initailise the sequence-to-sequence model for summarisation task using the pytorch framework.

In [48]:
## Load the fine-tuned model from the model hub
model_name = "arinze/t5_finetuned_xsum_hr"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [60]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline('summarization', model_name, framework="pt", max_length=1024)

We need to apply the model on chunks of Accountability comments

In [14]:
def chunks_joined(lst, n):
    """
    Divide a list of sentences into chunks of concatenated n sentences
    """
    all_chunks = []
    for i in range(0, len(lst), n):
        joined_sentences =  lst[i:i + n]
        joined_sentences = ' '.join(str(e) for e in joined_sentences)
        all_chunks.append(joined_sentences)
    return all_chunks

In [53]:
ys_accountability_corpus = list(ys_accountability['Q31_cleaned'])

We generate chucks each containing comments from 10 respondents.

In [55]:
ys_accountability_test_chunks  = chunks_joined(ys_accountability_corpus, 10)
print("Total number of chunks: ", len(ys_accountability_test_chunks))
ys_accountability_test_chunks[9]

Total number of chunks:  117


'Regularly decisions have to made by committee as no one person is accountable for anything. Trying to figure out who can make a call on anything as a result is difficult. as agents I believe we  need more control over  accounts and the credit limit for us needs to be raised as they is always checks made on agent to justify the credits given and this would also take a way time from mangers having to pick this up  as front line agents been able to offer adjustments that are needed on the account if they is a mistake from us  this would save on transfer to the billing rec team  Although we are one company we still work in silos sometimes.  Trying to get people to take ownership and accountability for anything can be very challenging and timewasting.  Time wasted trying to find the right person to speak to. the basic are missed no one is held to account and takes ownership any more Was torn between accountability and processes and procedures. Whilst a think processes and procedures play a

We can also know the total length of tokens in a chunk

In [59]:
print("Token of tokens in chunk 9 is:", len(tokenizer(ys_accountability_test_chunks[9])['input_ids']) + 2) # Plus 2 factors in the the start of sequence [CLS] and separator [SEP] tokens.

Token of tokens in chunk 9 is: 468


Generate summaries for all chunks

In [62]:
num_of_threads = 30
torch.set_num_threads(num_of_threads) #set number of threads according to cores of VM instance

Sequences longer than the max allowable length for the model are truncated to the max allowable length

In [81]:
all_acc_summaries = []
for i in tqdm(range(len(ys_accountability_test_chunks))):
    token_max_len = len(tokenizer(ys_accountability_test_chunks[i])['input_ids']) + 2
    acc_summary = summarizer(ys_accountability_test_chunks[i], max_length=token_max_len)
    # Take care of warning message about indices sequence length
    # "Token indices sequence length is longer than the specified maximum sequence length for this model (574 > 512). Running this sequence through the model will result in indexing errors"
    if token_max_len > tokenizer.model_max_length:
        acc_summary = summarizer(ys_accountability_test_chunks[i], max_length=tokenizer.model_max_length)
        
    all_acc_summaries.append(acc_summary)

100%|██████████| 117/117 [01:57<00:00,  1.00s/it]


In [85]:
all_acc_summaries_cleaned = []
## remove the "summary_text" prefix and convert the resulting list of dictionaries to strings
for i in range(len(all_acc_summaries)):
    sentence_l = [d['summary_text'] for d in all_acc_summaries[i]]
    all_acc_summaries_cleaned.append(" ".join(str(x) for x in sentence_l))

In [86]:
## Process results into a dataframe
ys_accountability_summary = pd.DataFrame({'YS Accountability Comment':ys_accountability_test_chunks, 'Summary': all_acc_summaries_cleaned})
display(HTML(ys_accountability_summary.head().to_html()))

,YS Accountability Comment,Summary
0,We need clearer RR paths to achieve the desired outcome. In general its not great at BT. Good example however of where this is done well is the Privacy Impact Assessment Process. Clear accountabilities within and across functions would enable greater speed of changedelivery accountability on other departments and teams. Always down to the SD manager to chase feedback and challenged on things that are out of their control. Getting things done requires too many sign offs and too many hoops to jump through. Its not clear how our work is valued across the organisation. Decisions are made at a senior level without proper thought of how we actually achieve the goals set with the resource provided. It then becomes the blame game when it goes wrong. There needs to be more discussion around fundamental decisions with the boots on the ground. Quite often processes fall to the same people as not enough people have the knowledge experience or time to take any responsibility for change. Bringing new people into change projects leaving current contingent of staff without ownership or proper control slows down progress. Legacy staff do have the know how and would like to see positive change and arent always the cause of needing it in the first place. I feel that patch managers are the only people accountable when it comes to control wayleaves or planning for example there is a huge lack of accountability which makes life tougher than it has to be. We can provide better service if we fix this. Accountability in the fact of other companys putting in our fibre network and half arseing it Putting it in wrong poorly and not up to our standard to which we have to then go and sort out and figure out what the hell it is theyve done Wasting more time and money. There is no accountability for individuals within the group. Almost all problems that i encounter are passed on to service delivery. Trying to get other areas of the business to take ownership is neigh on impossible,Changedelivery is not a good example of how it is done in BT but it is a great example of what we need to do.
1,I find regularly when you need to approach a team for support beyond your own immediate stakeholders they are reluctant to help or give time to assist on someone that is not related to their main workstack Some engineers need to take more ownership and accountability aswell as departments taking accountability of jobs so theyre not creating unnecessary truck rolls. I would have selected more categories for context it is a combination of processes and procedures which are bloated and long winded lack of accountability in ownership of tasks and actions and legacy systems that are not fit for business. there are people that do not put the customer first not going above and beyond the job role to progress work very frustrating not my job attitude in certain departments. There is no real accountability for other LOBs. For example SD own the tails however there is no great drive from FND and other departments to deliver a solution at pace and it takes the SD manager to actively drive things to get solutions. This takes time away from spending time in the field. Other LOBs and contractors need to share the accountability that SD managers take I feel we sometimes would rather make no decision as opposed to running the risk of making a wrong decision. too many departments dont take responsibility for what is their own jobs act with any speed to rectify issues...I.E CBRE the fibre teams all which seem to have a pass not to act on any escalations with any haste there needs to be a more structured procedure for agent feedback terrible mistakes are made on accounts and no one is held accountable for them. i feel that the company is too large and difficult to interact with other departments we are constantly highlighting problems with our processes and procedures to management and nothing ever gets done.constantly using work arounds to make systems 

In [87]:
display(HTML(ys_accountability_summary.tail().to_html()))

In [88]:
ys_accountability_summary.to_csv("Summary_for_Comments_in_YS_Accountability_Category.csv", index=False)

## Summarise all categories

In [16]:
## Load the fine-tuned model from the model hub
model_name = "arinze/t5_finetuned_xsum_hr"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline('summarization', model_name, framework="pt", max_length=1024)

In [22]:
ys_cat = ys_categories

In [50]:
ys_cat[2:]

['Collaboration',
 'Processes and procedures',
 'Working environment & culture',
 'Communication',
 'Leadership support',
 'Development in role',
 'Working relationships',
 'Other',
 'Manager support',
 'Wellbeing support']

In [51]:
ys_categories = ys_cat[2:4]

In [49]:
ys_cat[3]

'Processes and procedures'

In [52]:
ys_categories    # 'Processes and procedures' error

['Collaboration', 'Processes and procedures']

In [ ]:
# ToDo: use nltk to extract actual sentences rather than assume that what people entered were full sentences.

In [53]:
for category in ys_categories:
    ys_category = ys_data[ys_data['Q30'] == category]
    ys_category = ys_category[~ys_category['Q31'].isnull()]
    ys_category['Q31_cleaned'] = ys_category['Q31'].apply(lambda x:clean_text(x))
    ys_category_corpus = list(ys_category['Q31_cleaned'])
    ys_category_test_chunks  = chunks_joined(ys_category_corpus, 10)

    all_category_summaries = []
    print("Summarising ",category, "category...")
    for i in tqdm(range(len(ys_category_test_chunks))):
        token_max_len = len(tokenizer(ys_category_test_chunks[i])['input_ids']) + 2
        category_summary = summarizer(ys_category_test_chunks[i],min_length = 10, max_length=token_max_len) #min_length of 10 to take care of very short comments
        # Take care of warning message about indices sequence length
        if token_max_len > tokenizer.model_max_length:
            category_summary = summarizer(ys_category_test_chunks[i], max_length=tokenizer.model_max_length)
            
        all_category_summaries.append(category_summary)

    all_category_summaries_cleaned = []
    ## remove the "summary_text" prefix and convert the resulting list of dictionaries to strings
    for i in range(len(all_category_summaries)):
        cleaned_summaries = [d['summary_text'] for d in all_category_summaries[i]]
        all_category_summaries_cleaned.append(" ".join(str(x) for x in cleaned_summaries))

    ys_category_summary = pd.DataFrame({'YS_' + category + '_Comment':ys_category_test_chunks, 'Summary': all_category_summaries_cleaned})
    ys_category_summary.to_csv('Summary_for_Comments_in_YS_' + category + '_Category.csv', index=False)

#def summarise_categories(list_of_categories):
#

Summarising  Collaboration category...


100%|██████████| 97/97 [04:37<00:00,  2.86s/it]


Summarising  Processes and procedures category...


100%|██████████| 821/821 [41:59<00:00,  3.07s/it]  


In [54]:
## Load the fine-tuned model from the model hub
model_name = "arinze/t5_finetuned_xsum_hr"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
summarizer = pipeline('summarization', model_name, framework="pt")

def summarise_categories(ys_data_path = '../yoursay_02_2022.csv'):
    """
    Summary of Comments from YS survey using a fine-tuned summarisation model.

    Parameter: CSV file of YS survey

    Returns: Individual dataframes of respondents' comments and their corresponding summaries
    as individual files per category
    
    """

    ys_data = pd.read_csv(ys_data_path)
    ys_data.drop([0], inplace=True)
    ys_categories = list(ys_data['Q30'].unique())
    ys_categories = [x for x in ys_categories if str(x) != 'nan'] #remove nan from the list
    for category in ys_categories:
        ys_category = ys_data[ys_data['Q30'] == category]
        ys_category = ys_category[~ys_category['Q31'].isnull()]
        ys_category['Q31_cleaned'] = ys_category['Q31'].apply(lambda x:clean_text(x))
        ys_category_corpus = list(ys_category['Q31_cleaned'])
        ys_category_test_chunks  = chunks_joined(ys_category_corpus, 10)

        all_category_summaries = []
        print("Summarising ",category, "category...")
        for i in tqdm(range(len(ys_category_test_chunks))):
            token_max_len = len(tokenizer(ys_category_test_chunks[i])['input_ids']) + 2 # Plus 2 factors in the the start of sequence [CLS] and separator [SEP] tokens.
            category_summary = summarizer(ys_category_test_chunks[i], min_length = 10, max_length=token_max_len) #min_length of 10 to take care of very short comments
            # Take care of warning message about indices sequence length
            if token_max_len > tokenizer.model_max_length:
                category_summary = summarizer(ys_category_test_chunks[i], max_length=tokenizer.model_max_length)
                
            all_category_summaries.append(category_summary)

        all_category_summaries_cleaned = []
        ## remove the "summary_text" prefix and convert the resulting list of dictionaries to strings
        for text in range(len(all_category_summaries)):
            cleaned_summaries = [d['summary_text'] for d in all_category_summaries[text]]
            all_category_summaries_cleaned.append(" ".join(str(x) for x in cleaned_summaries))

        ys_category_summary = pd.DataFrame({'YS_' + category + '_Comment':ys_category_test_chunks, 'Summary': all_category_summaries_cleaned})
        ys_category_summary.to_csv('Summary_for_Comments_in_YS_' + category + '_Category.csv', index=False)

#def summarise_categories(list_of_categories):
    

#DONE: wrapper function to summarise all categories.
#TODO: fine-tune another pre-trained model for summarisation task and apply to comments


## Communication Category

In [2]:
import pandas as pd

comms = pd.read_csv("C:\\Users\\614246757\OneDrive - BT Plc\\Documents\\HR Analytics\hr-text-analytics\\Summary_for_Comments_in_YS_Communication_Category.csv")

In [3]:
from IPython.display import display, HTML

In [4]:
display(HTML(comms.head().to_html()))

,YS_Communication_Comment,Summary
0,There are a lot of different streams working in Silo and although communication has improved a little it could be even better no company wide emails about changes . chinese whispers Communication during the recruitment process was absolutely shambolic and disgusting the way HR communicated certain individual results. This could have potentially caused serious damage to mental health issues. This needs to be addressed going forward at a higher level communication between different LOBs is non existent making jobs more difficult In my role specifically within Job Control it is really difficult to get certain teams in planning to contact myself and to get tasks done within a decent timeframe. I find even though we are a communications company inside the company we have poor communications within our own teams staff to manager or staff to other members of staff including Traffic ManagementCivils organising a hoist. Communication in house as well as communication with external customers seems to be constant miscommunication which leads to lots of wasted time and money. Makes us look unprofessional Lack of communication between different departments get to jobs and waiting round for update for example TM and no TM turns up wasting time and money Lots of jobs passed from dept to dept and no communication between them and enduser has to wait as a result There are often times we will arrive to a site or job only to discover that previous work is wholly or partially incomplete. Communication is lacking in key areas such as the status of A55s OCR work etc.,There is a lack of communication between different LOBs within a company in Silo which could have caused serious damage to mental health.
1,for a communications company nobody talks to eavh other Communication between managers FBCs site contacts can be improved. We are sometimes sent to jobs that cant be completed or are already completed. Also sent to different patches to help and are not given enough information to complete the jobs at hand. Partners are doing large jobs but are leaving us with no information about what theyve done and not done Having just moved into a new role that involves being the middle person between OR and our partners I can sometimes spend half my day just trying to find what part of the business I need to contact or refer someone too. departments do not communicate with each other Poor communication with service providers leading to jobs being re reported without end users knowledge and wasted truck rolls. Sometimes decisions from above do not get passed down and explained clearly. I feel as though the communication within all levels of the company could be improved. Simple things like trying to find out an item code for a piece of equipmenttool can be difficult to obtain. coms from one business unit to another is poor Senior managers often dont know whats going on. Important messages are usually received by rumour. different areas in the country work differently however very little experiences are shared,Communication between a company and partners can be improved if they are not given enough information to complete their jobs.
2,getting replies via emailsmessaging takes for ever sometimes Ie field insights should be called less as if we had an online portal we could upload A55 to be vetted without having to waste time calling field insights FOR A COMMUNICATIONS COMPANY WE ARE RUBBISH AT COMMUNICATING. WE SEEM TO MAKE IT DIFFICULT FOR END USERS TO CONTACT US ONLINE SEEMS THE ONLY WAY WHEN TELEPHONES POST AND EMAIL DO EXIST INTERNALLY THERRE SEEMS TO BE NO COMEBACK WHEN OTHER DEPARTMENTS CANNOT OR WILL NOT HELP For example. Jobs being sent back out in workstack when job has already been completed which has an impact on efficiency I selected communication because I feel like sometimes change is not communicated effectively across the organisation. The managers do not communicate to the clerk of works which stops progress as

## Leadership Category

In [5]:
leadership = pd.read_csv("C:\\Users\\614246757\OneDrive - BT Plc\\Documents\\HR Analytics\hr-text-analytics\\Summary_for_Comments_in_YS_Leadership support_Category.csv")

In [6]:
display(HTML(leadership.head().to_html()))

,YS_Leadership support_Comment,Summary
0,Manager and patch leads coil help more rather than pass the buck I feel there is a blame culture The only time I hear from my manager is when I have made a mistake. I could do a 100 jobs but if I get 1 wrong on the tracker I get a phone call. Im not a robot and mistakes can happen when the pressure is on but instead of phone calls about mistakes how about praise for doing the 99 correctly too many changes are happening which again are fine but there needs to be a balance. Its just seems leadership teamstransformation teams just want to put a tick in the box and we have to go with it our manager has not got a clue about what we do as engineers he has come from an external company and manages us as a spread sheet not as people. he has no idea that a socket change is not as complicated as an underground fault. he also has no interest in finding out what it is that we actually do for a living. Lack of support from leaders we are just expected to put our heads down and do what we are told. No faith in the top management I dont feel that the leadership in my department is inspiring or very helpful. HOW CAN YOU EXPECT TEAM MEMBERS TO BE FOCUSED WHEN THE THREAT OF REDUNDANCY AND OFFICE CLOSURES IS HANGING OVER PEOPLES HEADS. I leaders are not engineers they are computer warrior who have no experience at the job THERE IS NO LEADERSHIP ALL THEY WANT TO DO IS TO GET RID OF ALL EXPERIENCED EMPLOYEES. THERE IS NO HONEST CONVERSATIONS AND IT WOULD SEEM THERE IS AN ULTERIOR MOTIVE TO EVERYTHING THEY DO,I am not a robot but if I get a phone call from a manager I feel there is a blame culture in my department.
1,I selected leadership support i feel it would be unwise to explain in detail on this form our middle management are changing the goal posts on a daily basis and at the moment we dont know if we are coming or going moving engineers between teams is not sorting the problem when the engineers are not skilled to do the work that needs to be done. Since the move to patch management Newsites has been given a back seat to Ethernet.We need an extra FBC in my patch as the workload is far to big for just two and double other areas FBCs I have over 70 sites and other FBCs have 40.I am being asked to move FODs to fiddle the figures so we dont have a site fail. The turn over of managers doesnt help and as my new one has only been on one week I cant score him on this survey. Incompetence is demonstrated at every single level. Everyones in it for themselves. what do our patch leads actually do Apart from van checks you never see or hear from them. Whatever is happening in the background it hasnt resulted in any coaching. You get better information by asking colleagues for help. Ive never once had a phone call off the manager asking how things are going either. I am very proud to work for Openreach and I love my job but I dont have any support within my team when I went live I was constantly made to feel stupid my buddying was shocking on most day I was suffering anxiety and dreading going to work in the morning because to much pressure on our heads about com jobs and hitting targets and not having any support to help me but now I have got a little bit better there is still no support at all has got a lot better now we have senior engineers but all my support comes from them now. I dont feel valued. it seems all desk based functions are now short staffed or staffed by inexperienced people and are not functioning adequately SL team need to take the time to understand the types of change that they are forcing on others. they are not practicing what they preach making working at BT miserable No faith in the leadership lost touch of reality dont want to know attitude disgusted at the way they treat the work force The place is full of bright promises nothing ever gets delivered when management want something they become your best friend. otherwise you are 6XXXXXXXX,I have been asked to move FODs in my patch to fiddl